Date: April 4th, 2022
 
Author: Nana

## Cloning Github, Mounting Drive

In [2]:
import pickle
import json
import random
import numpy as np
import pandas as pd
import getpass, os
import sys
username = getpass.getpass("Github username: ")
token = getpass.getpass("Github token: ")
os.environ["CRED"] = f"{username}:{token}"
os.environ["USR"] = f"{username}"
!git clone https://${CRED}@github.com/${USR}/2022Fintech-project
del username, token

Github username: ··········
Github token: ··········
Cloning into '2022Fintech-project'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 32 (delta 10), reused 21 (delta 6), pack-reused 0
Unpacking objects: 100% (32/32), done.


github ref: https://stackoverflow.com/questions/59454990/how-to-push-from-colab-to-github

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## LCS

In [11]:
# LCS: English/Chinese 
# ref: https://www.techiedelight.com/longest-common-substring-problem/

def LCS(X, Y):
    m, n = len(X), len(Y)
    maxLength = 0           # stores the max length of LCS
    endingIndex = m         # stores the ending index of LCS in `X`
 
    # `lookup[i][j]` stores the length of LCS of substring `X[0…i-1]` and `Y[0…j-1]`
    lookup = [[0 for x in range(n + 1)] for y in range(m + 1)]
    # fill the lookup table in a bottom-up manner
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if X[i - 1] == Y[j - 1]:
                lookup[i][j] = lookup[i - 1][j - 1] + 1
                # update the maximum length and ending index
                if lookup[i][j] > maxLength:
                    maxLength = lookup[i][j]
                    endingIndex = i
    # return longest common substring having length `maxLength`
    return X[endingIndex - maxLength: endingIndex]

In [49]:
brand_lexics_fp = '/content/2022Fintech-project/data/brand_lexics.json'
with open(brand_lexics_fp, 'r') as f:
  brand_lexics = json.load(f)
# train set內原有的brand names
train_brands = list(brand_lexics.keys())

In [ ]:
# merging set...?

In [50]:
# hyperparam
threshold, threshold_strict = 4, 6
B = len(train_brands)
lcs_mat = [[0 for _ in range(B)] for _ in range(B)]
# all(u'\u4e00' <= c <= u'\u9fff' for c in name.decode('utf-8'))
#######
from collections import defaultdict
four_lcs = defaultdict(set)
for i in range(len(train_brands)):
  for j in range(i, len(train_brands)): # O(len(train_brands)^2)
    a, b = train_brands[i], train_brands[j]
    # caching 
    lcs = lcs_mat[i][j]
    if lcs == 0:
      lcs = LCS(a, b) # O(mn)
    # clean spaces 
    lcs = lcs.replace(' ', '')
    lcs_mat[i][j] = lcs  # i < j 
    # threshold: stricter for strings with only English/digits
    curr_threshold = threshold
    if all( 48 <= ord(c) <= 122 for c in lcs): # english, digits (assuming brand names w/o puncts)
        curr_threshold = threshold_strict
    if len(lcs) >= curr_threshold:
        four_lcs[lcs].add(a)
        four_lcs[lcs].add(b)

In [51]:
len(four_lcs)

2839

In [53]:
reps = [(k,v) for k, v in four_lcs.items() if len(v) > 1]
len(reps)

312

In [54]:
reps

[('adidas', {'adidas', 'adidas 愛迪達', '愛迪達 adidas'}),
 ('MENTHOLATUM', {'MENTHOLATUM 曼秀雷敦', '曼秀雷敦 MENTHOLATUM'}),
 ('國際牌P', {'國際牌 PANASONIC', '國際牌 Panasonic'}),
 ('Panasonic', {'Panasonic', '國際牌 Panasonic'}),
 ('anasonic', {'Panasonic', '國際牌 Panasonic', '國際牌 panasonic'}),
 ('BALANCE', {'B BALANCE', 'NEW BALANCE'}),
 ('osmetics',
  {'Bomb Cosmetics',
   'First Cosmetics Works de第一化粧品',
   'essence cosmetics 艾森絲',
   'theBalm Cosmetics',
   'vt cosmetics',
   'vt cosmetics X BT21'}),
 ('cosmetics',
  {'essence cosmetics 艾森絲', 'vt cosmetics', 'vt cosmetics X BT21'}),
 ('Cellina', {'Cellina 雪芙蘭', '雪芙蘭 Cellina'}),
 ('Bioré', {'Bioré', "MEN'S Bioré", '蜜妮 Bioré'}),
 ('LION獅王', {'LION 獅王', 'LION獅王'}),
 ('KISSME', {'KISSME', 'KISSME 奇士美'}),
 ('特福Tefal', {'法國特福 Tefal', '特福 Tefal'}),
 ('KOSE高絲', {'KOSE 高絲', 'KOSE 高絲-專櫃'}),
 ('ME蘭蔻', {'LANCOME 蘭蔻', 'LANCÔME 蘭蔻'}),
 ('eHouse', {'Bonnie House 植享家', 'Etude House'}),
 ('理膚寶水', {'LA ROCHE POSAY 理膚寶水', '理膚寶水'}),
 ('統一瑞穗', {'統一瑞穗', '統一瑞穗極制'}),
 ('飛利浦P', {

In [61]:
import csv
datadir = '/content/drive/MyDrive/FinTech-final-project/data'
with open(f'{datadir}/raw_rep_brand_names.csv', 'w') as f:
    # create the csv writer
    writer = csv.writer(f)
    header = ['lcs', 'brand_name_1', 'brand_name_2', 'brand_name_3', 'brand_name_4',
              'brand_name_5', 'brand_name_6', 'brand_name_7', 'brand_name_8', 'brand_name_9'
              ,'brand_name_10', 'brand_name_11', 'brand_name_12']
    # write a row to the csv file
    writer.writerow(header)
    for (k, v) in reps:
      row = [k]
      row.extend(list(v))
      writer.writerow(row)

## import difflib 

## Pushing the notebook 

In [5]:
#Copy file from either google drive after mounting using file browser
!cp /content/drive/MyDrive/FinTech-final-project/3.0-brand-names-multi-lexicalizations.ipynb /content/2022Fintech-project/codes

In [9]:
!git config --global user.email "b06102020@ntu.edu.tw"
!git config --global user.name "Nana2929"
%cd /content/2022Fintech-project
!git add /content/2022Fintech-project/codes/3.0-brand-names-multi-lexicalizations.ipynb
!git commit -m 'brand-names-checking'  # commit in Colab
!git push

/content/2022Fintech-project
[main 74ca50f] brand-names-checking
 1 file changed, 1 insertion(+)
 create mode 100644 codes/3.0-brand-names.ipynb
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.43 KiB | 1.43 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Nana2929/2022Fintech-project
   b0b5f03..74ca50f  main -> main
